In [ ]:
#install packages
!pip install ttach
!pip install git+https://github.com/gbaydin/hypergradient-descent.git
!pip install pretrainedmodels
!pip install --upgrade efficientnet-pytorch

In [ ]:
#Imports
import os
import sys
import glob
import torch
import torchvision

import numpy    as np
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser


import pandas as pd

from hypergrad import SGDHD, AdamHD

import pretrainedmodels

from efficientnet_pytorch import EfficientNet

import ttach as tta
import pandas as pd
import shutil

%matplotlib inline

In [ ]:
NAME = "SUBMISSION"

MODEL_NAME1 = 'se_resnext101_32x4d' # could be fbresnet152 or inceptionresnetv2

DIM_1 = 550
DIM_2 = 500
DIM_TEST_1 = 550
DIM_TEST_2 = 500


BATCH_SIZE = 8

NUM_EPOCHS1 = 10
extra=50

random_seed = 42
shuffle_dataset = True
validation_split = .1

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
!mkdir ~/.kaggle

# copy the newly-uploaded file to that directory
! cp /kaggle/input/kaggle/kaggle.json ~/.kaggle/

# # change permission of the file
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import os
# create new directory for the Kaggle project
!mkdir Cassava_Disease_Classification
os.chdir('Cassava_Disease_Classification')

In [ ]:
# install new Kaggle API
!pip install --upgrade pip
!pip install kaggle==1.5.6

In [ ]:
!kaggle competitions download -c ammi-2021-convnets

In [ ]:
!unzip ammi-2021-convnets.zip > /dev/null

In [ ]:
classes=['cbsd', 'healthy', 'cbb', 'cmd', 'cgm']

In [ ]:
#pseduo labled data
labeled_df= pd.read_csv('/kaggle/input/labeled/labelednew.csv')

In [ ]:
predictnew=labeled_df.to_numpy()
pred=np.array(predictnew)
unlabled_list =pred[(-pred[:,1].astype('float64')).argsort()[:extra]]
for unlabeled in unlabled_list:
    shutil.copy(unlabeled[2],'/kaggle/working/Cassava_Disease_Classification/train/train/'+classes[int(unlabeled[0])] )

In [ ]:
data_path = "."
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages")

In [ ]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(DIM_1),
                                       transforms.RandomCrop(DIM_2),
                                       transforms.RandomHorizontalFlip(0.3),
                                       transforms.RandomVerticalFlip(0.3),
                                       transforms.ToTensor(),
                                       transforms.RandomErasing(0.1),
                                       transforms.Normalize(mean=mean, std=std)])

test_transforms = transforms.Compose([ transforms.Resize(DIM_TEST_1),
                                      transforms.CenterCrop(DIM_TEST_1),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=mean, std=std)])

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.dim = dim

        self.targets = []
        

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.targets.append(i)
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.targets.append(i)
                
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, self.dim, self.dim), classCategory

class CassavaTestDataset(Dataset):
    def __init__(self, path, dim, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.indices = []
        self.dim=dim

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
                self.indices.append(fileName.split("/")[-1])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        index = self.file_list[idx][2]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, self.dim, self.dim), index

train_data = CassavaDataset(train_path, dim=DIM_2, transform=train_transforms)
test_data = CassavaTestDataset(test_path, dim=DIM_TEST_1, transform=test_transforms)
unlabled_data = CassavaTestDataset(extraimage_path, dim=DIM_TEST_1, transform=test_transforms)

In [ ]:
# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                             sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
unlabled_loader = torch.utils.data.DataLoader(unlabled_data, batch_size=BATCH_SIZE)

In [ ]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    model.eval()
    # We do not need to maintain intermediate activations while testing.
    accs = []
    with torch.no_grad():
        
        # Loop over test data.
        for features, target in data_loader:
          
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct = pred.cpu().eq(target.view_as(pred)).sum().item()
            total = pred.shape[0]
            accs.append(correct/total)

    # Print test accuracy.
    percent = 100. * np.mean(accs)
    st = np.std(accs)
    return percent, st

def train(model, criterion, data_loader, test_data_loader, optimizer, num_epochs, filename):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    best_acc = 0

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in range(num_epochs):
        
        # Loop over data.
        for batch_idx, (features, target) in enumerate(data_loader):

            # Forward pass.
            output = model(features.to(device))
            loss = criterion(output.to(device), target.to(device))

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01 

        # Print out progress the end of epoch.
        print('----- Model Evaluation -----')
        print('Epoch: {}/{} \tTrain Loss: {:.6f}'.format(epoch+1,num_epochs, ema_loss))
        # train_a, train_st = test(model,data_loader)
        test_a, test_st = test(model,test_data_loader)
        # print(f'Train accuracy: ({train_a:.2f}%) with std:({train_st:.2f})')
        print(f'Test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')
        if test_a > best_acc:
            best_acc = test_a
            torch.save(model.state_dict(), filename+".pth")
   
    checkpoint = torch.load(filename+".pth")
    model.load_state_dict(checkpoint)
    print("------")
    test_a, test_st = test(model,test_data_loader)
    print(f'Final test accuracy: ({test_a:.2f}%) with std:({test_st:.2f})')
    
    return model
    

def generate_predictions(model,data_loader):
    model.eval()
    preds=[]
    idx=[]

    print('----- MAKING PREDICTIONS -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, indices in data_loader:
            
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            for p,ind in zip(pred,indices):
                idx.append(ind)
                preds.append(p.item())

    return preds,idx

def map_to_classes(n):
    return train_data.classes[n]

In [ ]:
def get_resnext(model_name):
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.last_linear = nn.Linear(in_features=2048, out_features=5, bias=True)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamHD(model.parameters(), lr=1e-4, hypergrad_lr=1e-9)

    return model, criterion, optimizer

def get_eff_net(model_name, dim=1792):
    model = EfficientNet.from_pretrained(model_name)
    model._fc = nn.Linear(in_features=dim, out_features=5, bias=True)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = AdamHD(model.parameters(), lr=1e-4, hypergrad_lr=1e-9)
    
    return model, criterion, optimizer

In [ ]:
preds = []
def generate_predictions(model,data_loader):
    model.eval()
    preds=[]
    idx=[]
    values=[]

    print('----- MAKING PREDICTIONS -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for features, indices in data_loader:
            
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            value = output.max(dim=1, keepdim=True)
            for p,v,ind in zip(pred,value[0],indices):
#                 idx.append(ind)
#                 print(v[0])
                preds.append([p.item(),v.item(),ind])
    return preds

    

In [ ]:
model1, criterion, optimizer = get_resnext(MODEL_NAME1)

model1 = train(model1, criterion, train_loader, valid_loader, optimizer, num_epochs=NUM_EPOCHS1, filename="effnet")

In [ ]:
test(model1, train_loader)

In [ ]:
predictions=[]
tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))
predictions, _ = generate_predictions(tta_model.to(device),test_loader)
# preds.append(predictions)

In [ ]:
final_predictions = np.mean(preds,axis=0)
final_predictions.shape

ss = pd.DataFrame({
    "Category": predictions,
    "Id": test_data.indices
})

In [ ]:
ss.to_csv('resnet_submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c ammi-2022-convnets -f desparate.csv -m "last"

In [ ]:
ss["Category"] = predictions
ss["Category"] = ss["Category"].apply(map_to_classes)
ss.head()

## Pseudo labling

In [ ]:
model1, criterion, optimizer = get_resnext(MODEL_NAME1)
checkpoint = torch.load("/kaggle/input/bestmod/bestmod.pth")
model1.load_state_dict(checkpoint)
# tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))

In [ ]:
classes=['cbsd', 'healthy', 'cbb', 'cmd', 'cgm']
test(model1.to(device), train_loader)
predict= generate_predictions(tta_model.to(device),unlabled_loader)
classes=train_data.classes
pred=np.array(predict)

In [ ]:
labeled_df= pd.DataFrame(predict)
labeled_df.to_csv('./labelednew.csv', index=False)

In [ ]:
labeled_df= pd.read_csv('/kaggle/input/labeled/labelednew.csv')
predictnew=labeled_df.to_numpy()
pred=np.array(predictnew)
unlabled_list =pred[(-pred[:,1].astype('float64')).argsort()[:1000]]
for unlabeled in unlabled_list:
    shutil.copy(unlabeled[2],'/kaggle/working/Cassava_Disease_Classification/train/train/'+classes[int(unlabeled[0])] )

In [ ]:
shutil.make_archive('newtrain', 'zip', '/kaggle/working/Cassava_Disease_Classification/train/train')

### Testing

In [ ]:
final_predictions = np.mean(preds,axis=0)
final_predictions.shape

In [ ]:
ss = pd.DataFrame({
    "Category": final_predictions,
    "Id": test_data.indices
})
ss.head()

In [ ]:
ss["Category"] = predictions
ss["Category"] = ss["Category"].apply(map_to_classes)
ss.head()

In [ ]:
ss.to_csv(NAME+".csv",index=False)

In [ ]:
resnet, criterion, optimizer = get_resnext(MODEL_NAME1)

In [ ]:
checkpoint = torch.load("/kaggle/input/resent/resnext.pth")
resnet.load_state_dict(checkpoint)
tta_model = tta.ClassificationTTAWrapper(resnet, tta.aliases.five_crop_transform(DIM_TEST_2,DIM_TEST_2))
tta_model.to(device)
tta_model.eval()